# Plotting the CartPole experiment results

Experiements follow
Note that some were run in the terminal and thus do not show up here.

Ran the following 

python train_pg_f18.py CartPole-v0 -n 100 -b 1000 -e 3 -dna --exp_name sb_no_rtg_dna


python train_pg_f18.py CartPole-v0 -n 100 -b 1000 -e 3 -rtg -dna --exp_name sb_rtg_dna


python train_pg_f18.py CartPole-v0 -n 100 -b 1000 -e 3 -rtg --exp_name sb_rtg_na


python train_pg_f18.py CartPole-v0 -n 100 -b 5000 -e 3 -dna --exp_name lb_no_rtg_dna


python train_pg_f18.py CartPole-v0 -n 100 -b 5000 -e 3 -rtg -dna --exp_name lb_rtg_dna


python train_pg_f18.py CartPole-v0 -n 100 -b 5000 -e 3 -rtg --exp_name lb_rtg_na

## Comparison of different training algorithms

Run using:
python plot.py data/sb_no_rtg_dna_CartPole-v0_10-09-2019_17-26-25 data/sb_rtg_dna_CartPole-v0_10-09-2019_17-27-18 data/sb_rtg_na_CartPole-v0_11-09-2019_17-29-23

and same format for the larger batch size data.

#### Analysis
So we can see that the network using reward to go seems more solidly convergent on the target of 200 reward. 
Not normalizing the advantage actually helps it to converge faster, which makes sense, because the gradient will be larger, but the variance will also be higher I think, as the rewards that we scale the log probability of an action given a state are larger. This higher variance displays itself, as not Normalizing advantage results in a less thorough or complete convergence to the optimum. This is the case for both large and small batches.


The larger batches do not converge much faster if at all, which I believe makes sense, as we use tf.reduce_mean() for the loss function. This means that the gradients should be approximately the same magnitude. However, with larger batches, you expect the estimated gradient to be a better estimate of the true gradient. I Suspect this helps it to convert a little bit faster, and it helps the methods with higher variance converge better to the optimum. Because it seems to decrease the variance for every method, every method performs better at convergence.

### Plot of the small_batch trainings: Cart Pole
![Plot of the small_batch trainings: Cart Pole](../result_plots/Figure_1.png "Plot of the small_batch trainings: Cart Pole")


### Plot of the large_batch trainings: Cart Pole
![Plot of the small_batch trainings: Cart Pole](../result_plots/large_batch.png "Plot of the small_batch trainings: Cart Pole")


## Cart Pole demonstration!
The first one is training for 3 iterations of 500 batches per iteration. Not very much training. Not a great result.

The second one is after just 50 iterations of training at 500 batches per iteration. Note this is half the batch size as the above graphs. It's doing well. 

Both run like 

python train_pg_f18.py CartPole-v0 -n #_iterations -b 500 -e 1 -dna -rtg --exp_name test --save_models --render

So they both used reward to go and didn't normalize advantage

## Very Little Training
![SegmentLocal](../result_plots/cart_pole_v0_n3_b500.gif "segment")


## Slightly More Training
![SegmentLocal](../result_plots/cart_pole_v0_n50_b500.gif "segment")



# Continuous Inverted Pendulum
## Optimizing batch size
Ran using the bash script.

![Plot of the small_batch trainings: Cart Pole](../result_plots/Another-plot-of-batch-sizes-with-lr-of-5e-3Figure_1.png "Plot of the small_batch trainings: Cart Pole")

My theory is that when they hit the really high reward they all of a sudden get a large gradient, because the gradient is grad(logprob(action))*reward , and that reward will get really big. This large gradient will then result in a large mistep in a wrong direction, and it then it's catastrophically wrong. Maybe.

Ah, but it should be normalizing the return... I think it is, so it should be that large. The step size should be about the same!

Yet see below. This theory must be wrong. This is using even higher learning rate, and you have better results.
Maybe it's just drunken enough to quickly stumble into favorable setups.
IDK though. You still see the catastrophe, but perhaps it just gets back to the good stuff faster:)

### Probably wrong Theory about it's bad learning
Perhaps minimizign -log_prob(action|state) is fine if you're going for maximum likelihood model.
But maybe minimizing -log_prob(a|s)\*(reward) isn't good, as you're then subsequently trying to make the reward smaller? IDK. Actually that probably doesn't matter too much.

I wonder if the reward going negative is a real problem here. Like with rewards allowed to go negative, you are reversing the sign of log_prob(a|s) and then you should be moving in the opposite direction. Wait, actually we want that for the case of (-) reward [we want to go in the opposite direction in that case]. So that definitely isn't a problem.

Empirically, I should try to maximize the gradient in tensorflow just to make certain.

#### Better thought:
I think this is catastrophic forgetting that reddit talks about.



### Trying a faster learning rate
![Plot of the small_batch trainings: Cart Pole](../result_plots/faster_lr.tiff "Plot of the small_batch trainings: Cart Pole")

We see that many of the batch sizes reach the target, and all experience catastrophic forgetting. The smallest batch_size to reach was 2000, but 3k worked a lot faster. We'll go ahead and use 2k to be consistent with the prompt.

# Optimizing LR


![](../result_plots/lr_opt.tiff "")
So clearly only the lower learning rate actually worked, and it was quite unstable. I think it'll be a tradeoff there. Probably the stability merits further exploration.

## Demonstration of Continuous Task
Produced via:

(venv) Matthews-MacBook-Air-6:hw2 matthewhunt$ python train_pg_f18.py InvertedPendulum-v2 -ep 1000 --discount 0.9 -n 100 -e 1 -b 3000 -lr 5e-2 -rtg --exp_name test --script_optimizing_dir showing_invert_pend --save_models --save_best_model


After just 26 iterations of training, it hit the maximum reward and looked like:
(venv) Matthews-MacBook-Air-6:hw2 matthewhunt$ python train_pg_f18.py InvertedPendulum-v2 -ep 1000 --discount 0.9 -n 100 -e 1 -b 3000 -lr 5e-2 -rtg --exp_name test --run_model_only my_save_loc/showing_invert_pend/test_InvertedPendulum-v2_16-09-2019_18-36-27.ckpt --render

![SegmentLocal](../result_plots/continuous_gif.gif "segment")




### Testing whether we can reduce how the catastrophic forgetting looks if we do multiple experiments.

using 
python train_pg_f18.py InvertedPendulum-v2 -ep 1000 --discount 0.9 -n 100 -e 4 -b 3000 -lr 5e-2 -rtg --exp_name multi_experiment

This one used 4 experiments. The result is more what we'd expect to see. Several of them had reached the 1k mark, but we still see the catastrophic forgetting.

![](../result_plots/4-experiment plot.tiff "")

## Problem 6: NN-baselines. 
In this problem, we train a separate NN to learn a baseline which approximates the reward-to-go (Q function)

Let's see if it helps the cartpole training task.

Running 
python train_pg_f18.py CartPole-v0 -n 100 -b 1000 -e 4 -rtg --exp_name no_baseline_sb_rtg_na
python train_pg_f18.py CartPole-v0 -n 100 -b 1000 -e 4 -rtg --exp_name yes_baseline_sb_rtg_na --nn_baseline

The baseline loss gradually went down to about .3, and it's starting value was 0.5ish.

As you can see, it neither helps nor hurts for this simple task.

![](../result_plots/cartpole_baseline_comparison.tiff "")

## Lunar lander

Ran
python train_pg_f18.py LunarLanderContinuous-v2 -ep 1000 --discount 0.99 -n 100 -e 1 -l 2 -s 64 -b 40000 -lr 0.005 -rtg --nn_baseline --exp_name ll_b40000_r0.005 --save_models --script_optimizing_dir lunar_lander_best_models --save_best_model

This saves the best model from just one of the runs.
I cut the training off around 70ish to save time with my computer.
The output appears like. 
![](../result_plots/lunar_lander_3e.tiff "")



That model looks like this running:
![SegmentLocal](../result_plots/ll_gif.gif "segment")



## Half-cheetah Task

In this task we train mujuco's half-cheetah agent. 
First we ran a parameter search over several learning rates and batch sizes to determine which would be suitable for further experimentation. Following that we tested a successful lr/bs parameter set with different variance reduction techniques, such as NN-baselines and reward-to-go.

we performed the parameter search using the bash script: half_cheetah_optimizer.sh

Results as follows:
![](../result_plots/cheetah_big_triage.tiff "")

we see that generally smaller learning rates are more stable, and the larger batch-sizes do better. But there wasn't such a clear winner as I'd have expected. I went with batch size = 50000, lr = 1e-2.

Running several variations of variance reduction with this agent with the following commands:


python train_pg_f18.py HalfCheetah-v2 -ep 150 --discount 0.95 -n 100 -e 3 -l 2 -s 32 -b 50000 -lr 1e-2 --exp_name hc_b50000_r1e-2

python train_pg_f18.py HalfCheetah-v2 -ep 150 --discount 0.95 -n 100 -e 3 -l 2 -s 32 -b 50000 -lr 1e-2 -rtg --exp_name hc_b50000_r1e-2

python train_pg_f18.py HalfCheetah-v2 -ep 150 --discount 0.95 -n 100 -e 3 -l 2 -s 32 -b 50000 -lr 1e-2 --nn_baseline --exp_name hc_b50000_r1e-2

python train_pg_f18.py HalfCheetah-v2 -ep 150 --discount 0.95 -n 100 -e 3 -l 2 -s 32 -b 50000 -lr 1e-2 -rtg --nn_baseline --exp_name hc_b50000_r1e-2

Gave the following results

![](../result_plots/cheetah_big_runs.tiff "")

As expected, the network using reward to go and nn-baselines performed the best!



Now to actually see what one of these looks like, I ran one experiment only with baseline and rtg and saved the best model only.

This is it's graph
![](../result_plots/cheetah_single_train.tiff "")

Wow. That is a solid local minima.

![SegmentLocal](../result_plots/compressed_cheetah_gif.gif "segment")

Clearly that reward of ~50 isn't cutting it.
We should try multiple experiments and pick the best one. or just try again or for longer.


## Things to Do:
1. Optimize the baseline learning rate. Also run the pdb_baseline checker to see if it's actually normalizing. 
2. Make my game


In [ ]:
## Running the half-cheetah longer

This time I ran the half-cheetah for 500 training episodes, with a batch size of 50000:


The model is saved at maximum reward 
![](../result_plots/ "")

